## Импортирую библиотеки

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

## Установка начальных значений

In [2]:
money = 1000000
start = datetime.datetime(2015, 1, 1)
finish = datetime.datetime(2020, 12, 31)
short_window = 30
long_window = 90
stop_loss = 0.05  # 5% стоп-лосс
symbol = 'NKE'

## Загрузка данных по бумаге с учетом выбранного тикера

In [3]:
yf.pdr_override()
df = web.get_data_yahoo(symbol, start=start, end=finish)

[*********************100%***********************]  1 of 1 completed


## Отбираем из датафрейма данные OHLC

In [4]:
df = df[['Open', 'High', 'Low', 'Close']]

# Исходный датафрейм вида OHLC

In [5]:
df

,Open,High,Low,Close
Date,,,,
2015-01-02,48.275002,48.474998,47.055000,47.514999
2015-01-05,47.255001,47.275002,46.564999,46.750000
2015-01-06,46.945000,47.075001,46.035000,46.474998
2015-01-07,46.805000,47.650002,46.549999,47.435001
2015-01-08,47.830002,48.549999,47.810001,48.529999
...,...,...,...,...
2020-12-23,142.559998,143.600006,141.699997,141.759995
2020-12-24,141.100006,142.190002,141.100006,141.600006
2020-12-28,142.539993,142.919998,141.039993,142.429993


## Добавляю короткое скользящее окно и длинное скользящее окно

In [6]:
df['short_window'] = df['Close'].rolling(short_window).mean()
df['long_window'] = df['Close'].rolling(long_window).mean()

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\1811059243.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Также добавляю сигналы к покупке и продаже, заполняю np.nan, чтобы потом правильно отобразить сигналы на графике

In [7]:
df['sig_buy'] = np.where((df['short_window'].shift(1) <= df['long_window'].shift(1)) &
                         (df['short_window'] > df['long_window']), df['Close'], np.nan)
df['sig_sale'] = np.where((df['short_window'].shift(1) >= df['long_window'].shift(1)) &
                          (df['short_window'] < df['long_window']), df['Close'], np.nan)

# Датафрейм с добавленными окнами и сигналами

In [8]:
df

,Open,High,Low,Close,short_window,long_window,sig_buy,sig_sale
Date,,,,,,,,
2015-01-02,48.275002,48.474998,47.055000,47.514999,NaN,NaN,NaN,NaN
2015-01-05,47.255001,47.275002,46.564999,46.750000,NaN,NaN,NaN,NaN
2015-01-06,46.945000,47.075001,46.035000,46.474998,NaN,NaN,NaN,NaN
2015-01-07,46.805000,47.650002,46.549999,47.435001,NaN,NaN,NaN,NaN
2015-01-08,47.830002,48.549999,47.810001,48.529999,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2020-12-23,142.559998,143.600006,141.699997,141.759995,135.719333,126.242445,NaN,NaN
2020-12-24,141.100006,142.190002,141.100006,141.600006,136.184000,126.627222,NaN,NaN
2020-12-28,142.539993,142.919998,141.039993,142.429993,136.710333,127.005445,NaN,NaN


# Создание фрейма данных для отчета

In [9]:
result = pd.DataFrame(columns=['date', 'signal', 'num_shares', 'share_price', 'share_value', 'cash'])

In [10]:
num_shares = 0  # количество ценных бумаг
share_value = 0  # Текущая стоимость бумаг в портфеле
cash = round(money, 2)  # округляем начальные деньги до 2 знаков после запятой
# Переменные для хранения предыдущих значений 
prev_num_shares = 0
prev_share_value = 0
prev_cash = cash
stop_loss_triggered = False
last_buy_price = 0  #  переменная для хранения цены последней покупки

# Цикл для отображения сигналов покупки, продажи и стоп-лоссов (много предупреждений из-за appenda и pandas-data-reader)

In [11]:
for i in range(len(df)):
    row = df.iloc[i]
    next_day_row = df.iloc[i + 1] if i + 1 < len(df) else row

    # Проверка условий стоп-лосса
    if num_shares > 0: # стоп-лосс срабатывает только если у нас есть акции в портфеле
        stop_loss_price = last_buy_price * (1 - stop_loss)  # расчет stop_loss_price
        if row['Low'] < stop_loss_price:  # Цена упала ниже стоп-лосса
            sell_price = stop_loss_price if row['Close'] != stop_loss_price else row['Close'] #продаём по стоп-лоссу,если цена закрытия не равна цене стоп-лосса, если же цена закрытия равна стоп-лосу, то успеваем продать по закрытию
            sell_price = row['Open'] if row['Open'] < stop_loss_price else sell_price #продаём по цене открытия,если она ниже стоп-лосса т.е открытие торгов ниже нашгео критического значения
            cash = round(cash + sell_price * num_shares, 2) #считаем кэш
            share_value = 0 # акций нет => стоимость акций в портфеле ==0
            num_shares = 0 # акций нет => количество акций в портфеле ==0
            result = result.append({'date': row.name, 'signal': 'stop-loss', 'num_shares': prev_num_shares,
                                    'share_price': sell_price, 'share_value': share_value, 'cash': cash},
                                   ignore_index=True) #добавляем значения в итоговый датафрейм
        # Обновляем предыдущие значения
        prev_num_shares, prev_share_value, prev_cash = num_shares, share_value, cash

    # Обработка сигналов 'sig_buy' или 'sig_sale'
    if row['sig_buy'] > 0: #сигнал к покупке акций 
        result = result.append({'date': row.name, 'signal': 'sig_buy', 'num_shares': prev_num_shares,
                                'share_price': row['Open'], 'share_value': prev_share_value, 'cash': prev_cash},
                               ignore_index=True)  #добавляем значения в итоговый датафрейм
        if i + 1 < len(df):  # Если следующий день существует
            num_shares = cash // next_day_row['Open'] #покупаем акции на следующий день после сигнала к покупке по цене открытия
            share_price = next_day_row['Open'] # цена, по которой покупаем акции - цена открытия следующего торгового дня после сигнала к покупке
            share_value = round(num_shares * share_price ,2) # стоимость нашего портфеля
            cash = round(cash - share_price * num_shares, 2) # считаем остаток денег после покупки
            result = result.append({'date': next_day_row.name, 'signal': 'buy', 'num_shares': num_shares,
                                    'share_price': share_price, 'share_value': share_value, 'cash': cash},
                                   ignore_index=True)  #добавляем значения в итоговый датафрейм
            last_buy_price = share_price  # Обновляем цену последней покупки
            # Обновляем предыдущие значения
            prev_num_shares, prev_share_value, prev_cash = num_shares, share_value, cash

    elif row['sig_sale'] > 0 and prev_num_shares > 0: #сигнал к продаже работает только тогда, когда в портфеле есть акции
        sell_price = next_day_row['Open'] #продаём по цене следующего торогового дня после сигнала к продаже
        result = result.append({'date': row.name, 'signal': 'sig_sale', 'num_shares': prev_num_shares,
                                'share_price': row['Close'], 'share_value': prev_share_value, 'cash': prev_cash},
                               ignore_index=True) #добавляем значения в итоговый датафрейм
        if i + 1 < len(df):  # Если следующий день существует
            cash = round(prev_cash + sell_price * prev_num_shares, 2) #считаем наш кэш
            share_value = 0 # так как произошла продажа, то стоимость портфеля = 0
            num_shares = prev_num_shares # сколько акций у нас есть и ,соответственно, сколько продадим
            result = result.append({'date': next_day_row.name, 'signal': 'sale', 'num_shares': prev_num_shares,
                                    'share_price': sell_price, 'share_value': share_value, 'cash': cash},
                                   ignore_index=True) #добавляем значения в итоговый датафрейм
            num_shares = 0  # Обнуляем количество акций после продажи
        # Обновляем предыдущие значения
        prev_num_shares, prev_share_value, prev_cash = num_shares, share_value, cash

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\587234527.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\587234527.py:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\587234527.py:39: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\587234527.py:46: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\andre\AppData\Local\Temp\ipykernel_8252\587234527.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

# Сделаем количество акций целым числом, а не вещественным

In [12]:
result['num_shares'] = result['num_shares'].astype(int)

# Вывод итогового датафрейма 

In [13]:
result

,date,signal,num_shares,share_price,share_value,cash
0,2016-04-22,sig_buy,0,60.180000,0,1000000
1,2016-04-25,buy,16852,59.340000,999997.68,2.32
2,2016-04-29,sig_sale,16852,58.939999,999997.68,2.32
3,2016-05-02,sale,16852,59.090000,0,995787.0
4,2016-08-18,sig_buy,0,56.990002,0,995787.0
5,2016-08-19,buy,17348,57.400002,995775.23,11.77
6,2016-09-26,stop-loss,17348,54.530001,0,945998.24
7,2017-01-19,sig_buy,0,53.840000,0,945998.24
8,2017-01-20,buy,17775,53.220001,945985.52,12.72
9,2017-05-11,sig_sale,17775,54.240002,945985.52,12.72


# Расчет прибыли/убытка

In [14]:
last_sale_price = result.loc[result['signal'] == 'sale', 'share_price'].values[-1]
final_value = result['share_value'].values[-1] + result['cash'].values[-1]
initial_value = result.loc[result['signal'].isin(['sig_buy', 'sig_sale']), 'share_value'].values[0] + money
percentage_change = (final_value - initial_value) / initial_value * 100

# Вывод итоговых данных

In [15]:
print("Последняя цена продажи акций: ", round(last_sale_price, 2))
print("Итоговый капитал на момент последней продажи акций: ", round(final_value, 2))
print("Процент прироста/падения исходных вложений: ", percentage_change )

Последняя цена продажи акций:  75.5
Итоговый капитал на момент последней продажи акций:  978492.95
Процент прироста/падения исходных вложений:  -2.1507050000000043


# Визуализация, график позиций связан с верхним окном(большим графиком)

In [16]:
# График OHLC
trace_ohlc = go.Ohlc(x=df.index,
                     open=df['Open'],
                     high=df['High'],
                     low=df['Low'],
                     close=df['Close'],
                     name='OHLC')

# Графики скользящих средних
trace_short_window = go.Scatter(x=df.index,
                                y=df['short_window'],
                                mode='lines',
                                name='Короткое скользящее среднее')
trace_long_window = go.Scatter(x=df.index,
                               y=df['long_window'],
                               mode='lines',
                               name='Длинное скользящее среднее')

# Сигналы покупки, продажи и стоп-лосс
trace_sig_buy = go.Scatter(x=df[df['sig_buy'].notnull()].index,
                           y=df['sig_buy'].dropna(),
                           mode='markers',
                           marker=dict(symbol='triangle-up', color='green', size=15),
                           name='Покупка')
trace_sig_sale = go.Scatter(x=df[df['sig_sale'].notnull()].index,
                            y=df['sig_sale'].dropna(),
                            mode='markers',
                            marker=dict(symbol='triangle-down', color='red', size=15),
                            name='Продажа')
trace_stop_loss = go.Scatter(x=result[result['signal'] == 'stop-loss']['date'],
                             y=result[result['signal'] == 'stop-loss']['share_price'],
                             mode='markers',
                             marker=dict(symbol='triangle-down', color='black', size=15),
                             name='Стоп-лосс')

# Создание фигуры для первого окна и отображение графика
fig1 = go.Figure(data=[trace_ohlc, trace_short_window, trace_long_window, trace_sig_buy, trace_sig_sale, trace_stop_loss])
fig1.update_layout(title='График OHLC и скользящих средних',
                   yaxis=dict(title='Цена USD($)'),
                   xaxis=dict(title='Дата'))
fig1.show()